In [3]:
# 主CSV文件路径
main_csv_path = "/Users/yijingyang/Library/CloudStorage/OneDrive-个人/GradPilot/ProgramDB/AppliedDataScience/AppliedDataScience_df1.csv"

# 基础路径
base_path = "/Users/yijingyang/Library/CloudStorage/OneDrive-个人/GradPilot/ProgramDB/AppliedDataScience/fields_records"

# 保存结果
output_dir = "/Users/yijingyang/Library/CloudStorage/OneDrive-个人/GradPilot/ProgramDB/AppliedDataScience/汇总"

# 输出csv名字
merged_file_name = "美研-应用数据科学"
    

In [ ]:
import pandas as pd
import os
import glob

def integrate_fields_to_main_dataset():
    """
    将各个字段文件夹中的CSV数据按行索引直接添加到主数据集中
    自动发现每个文件夹中的CSV文件（假设每个文件夹只有一个CSV文件）
    
    Returns:
        pd.DataFrame: 整合后的数据集
    """
    
    # 读取主数据集
    print("📖 读取主数据集...")
    main_df = pd.read_csv(main_csv_path, encoding='utf-8-sig')
    print(f"主数据集包含 {len(main_df)} 个项目")
    
    # 字段映射配置：文件夹名 -> 要提取的字段列表（不再需要指定CSV文件名）
    field_configs = {
        '更多项目信息': ['更多项目信息'],
        '课程网址': ['课程网址'],
        '面试': ['面试'],
        # '其他字段': 跳过
        '申请费减免': ['申请费减免'],
        '所属院系（英文）': ['所属院系英文名称', '所属院系网址'],
        '推荐信': ['推荐信'],
        '托福送分ETS code': ['托福送分ETS code'],
        '项目标签': ['项目标签（英文）', '项目标签（中文）'],
        '职业项目': ['职业项目'],
        'Capstone或Thesis': ['毕业要求']
    }
    
    # 记录处理统计
    processing_stats = {
        'total_fields': len(field_configs),
        'successful_adds': 0,
        'failed_adds': 0,
        'field_details': {}
    }
    
    print("\n🔄 开始添加字段数据...")
    print("=" * 60)
    
    # 逐个处理每个字段
    for folder_name, target_columns in field_configs.items():
        print(f"\n📁 处理文件夹: {folder_name}")
        print(f"🎯 添加字段: {target_columns}")
        
        # 构建文件夹路径
        folder_path = os.path.join(base_path, folder_name)
        
        field_stats = {
            'folder_path': folder_path,
            'target_columns': target_columns,
            'folder_exists': False,
            'csv_found': False,
            'csv_path': None,
            'records_count': 0,
            'columns_added': []
        }
        
        try:
            # 检查文件夹是否存在
            if not os.path.exists(folder_path):
                print(f"   ❌ 文件夹不存在: {folder_path}")
                field_stats['error'] = "文件夹不存在"
                processing_stats['failed_adds'] += 1
                processing_stats['field_details'][folder_name] = field_stats
                continue
                
            field_stats['folder_exists'] = True
            
            # 自动发现文件夹中的CSV文件
            csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
            
            if not csv_files:
                print(f"   ❌ 文件夹中没有找到CSV文件: {folder_path}")
                field_stats['error'] = "没有找到CSV文件"
                processing_stats['failed_adds'] += 1
                processing_stats['field_details'][folder_name] = field_stats
                continue
            
            if len(csv_files) > 1:
                print(f"   ⚠️  警告: 文件夹中发现多个CSV文件，使用第一个: {csv_files}")
                print(f"   📋 所有CSV文件: {[os.path.basename(f) for f in csv_files]}")
            
            # 使用第一个找到的CSV文件
            csv_path = csv_files[0]
            csv_filename = os.path.basename(csv_path)
            field_stats['csv_found'] = True
            field_stats['csv_path'] = csv_path
            
            print(f"   📄 发现CSV文件: {csv_filename}")
            
            # 读取字段CSV文件
            print(f"   📖 读取文件...")
            field_df = pd.read_csv(csv_path, encoding='utf-8-sig')
            field_stats['records_count'] = len(field_df)
            print(f"   📊 找到 {len(field_df)} 条记录")
            
            # 检查行数是否匹配
            if len(field_df) != len(main_df):
                print(f"   ⚠️  警告: 行数不匹配! 主数据集: {len(main_df)}, 字段文件: {len(field_df)}")
                # 可以选择截断或填充，这里选择使用较短的长度
                min_length = min(len(main_df), len(field_df))
                print(f"   📏 使用最小长度: {min_length}")
            else:
                min_length = len(main_df)
                print(f"   ✅ 行数匹配: {min_length}")
            
            # 检查所需列是否存在
            available_columns = field_df.columns.tolist()
            missing_columns = [col for col in target_columns if col not in available_columns]
            
            if missing_columns:
                print(f"   ⚠️  缺失字段: {missing_columns}")
                print(f"   📋 可用字段: {available_columns}")
                # 只使用存在的字段
                existing_columns = [col for col in target_columns if col in available_columns]
                if not existing_columns:
                    print(f"   ❌ 没有可用的目标字段")
                    field_stats['error'] = "没有可用的目标字段"
                    processing_stats['failed_adds'] += 1
                    processing_stats['field_details'][folder_name] = field_stats
                    continue
                target_columns = existing_columns
            
            # 直接添加字段到主数据集
            print(f"   ➕ 添加字段: {target_columns}")
            added_columns = []
            
            for column in target_columns:
                if column in field_df.columns:
                    # 确保不会覆盖已存在的同名列
                    new_column_name = column
                    counter = 1
                    while new_column_name in main_df.columns:
                        new_column_name = f"{column}_{counter}"
                        counter += 1
                    
                    # 添加列数据（截断到最小长度）
                    main_df[new_column_name] = field_df[column].iloc[:min_length].reset_index(drop=True)
                    added_columns.append(new_column_name)
                    
                    if new_column_name != column:
                        print(f"     📝 {column} → {new_column_name} (重命名避免冲突)")
                    else:
                        print(f"     📝 添加: {column}")
            
            field_stats['columns_added'] = added_columns
            
            # 统计非空值
            non_null_stats = []
            for col in added_columns:
                non_null_count = main_df[col].notna().sum()
                non_null_stats.append(f"{col}: {non_null_count}/{len(main_df)}")
            
            print(f"   ✅ 成功添加 {len(added_columns)} 个字段")
            print(f"   📊 非空值统计: {', '.join(non_null_stats)}")
            
            processing_stats['successful_adds'] += 1
            
        except Exception as e:
            print(f"   ❌ 处理失败: {str(e)}")
            field_stats['error'] = str(e)
            processing_stats['failed_adds'] += 1
        
        processing_stats['field_details'][folder_name] = field_stats
        print("-" * 40)
    
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{merged_file_name}.csv")
    
    print(f"\n💾 保存整合结果...")
    main_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 文件已保存至: {output_path}")
    
    # 输出处理统计
    print(f"\n📊 整合完成统计:")
    print("=" * 60)
    print(f"总字段组数: {processing_stats['total_fields']}")
    print(f"成功添加: {processing_stats['successful_adds']}")
    print(f"添加失败: {processing_stats['failed_adds']}")
    print(f"最终数据集大小: {main_df.shape[0]} 行 × {main_df.shape[1]} 列")
    
    # 详细统计
    print(f"\n📋 详细处理结果:")
    for folder_name, stats in processing_stats['field_details'].items():
        status = "✅" if 'error' not in stats else "❌"
        print(f"{status} {folder_name}:")
        if 'error' in stats:
            print(f"     错误: {stats['error']}")
        else:
            print(f"     CSV文件: {os.path.basename(stats['csv_path']) if stats['csv_path'] else 'N/A'}")
            print(f"     记录数: {stats['records_count']}")
            print(f"     添加的列: {stats['columns_added']}")
    
    # 检查最终数据质量
    print(f"\n🔍 最终数据质量检查:")
    print(f"数据集形状: {main_df.shape}")
    print(f"列名: {list(main_df.columns)}")
    
    # 显示每列的非空值统计
    print(f"\n📈 各字段覆盖率:")
    total_records = len(main_df)
    
    # 只显示新添加的字段的统计
    original_columns = ['大学排名', '大学名称', '大学英文名称', '所在城市', '专业中文名称', '学位', 
                       '专业英文名称', '所属院系', '专业领域', '课程长度', '申请费（美元)', '开学期', 
                       '截止日期', 'GPA', '托福', '雅思', 'GRE', 'GMAT', '学术背景', '材料要求', 
                       '招生网址', '专业网址']
    
    new_columns = [col for col in main_df.columns if col not in original_columns]
    
    for column in new_columns:
        non_null_count = main_df[column].notna().sum()
        coverage = (non_null_count / total_records) * 100
        print(f"   {column}: {non_null_count}/{total_records} ({coverage:.1f}%)")
    
    return main_df, processing_stats

# 运行整合函数
if __name__ == "__main__":
    final_df, stats = integrate_fields_to_main_dataset()
    print(f"\n🎉 整合完成! 最终数据集形状: {final_df.shape}")

📖 读取主数据集...
主数据集包含 121 个项目

🔄 开始添加字段数据...

📁 处理文件夹: 更多项目信息
🎯 添加字段: ['更多项目信息']
   📄 发现CSV文件: 更多项目信息_提取结果.csv
   📖 读取文件...
   📊 找到 121 条记录
   ✅ 行数匹配: 121
   ➕ 添加字段: ['更多项目信息']
     📝 添加: 更多项目信息
   ✅ 成功添加 1 个字段
   📊 非空值统计: 更多项目信息: 110/121
----------------------------------------

📁 处理文件夹: 课程网址
🎯 添加字段: ['课程网址']
   📄 发现CSV文件: 课程网址_规则处理结果.csv
   📖 读取文件...
   📊 找到 121 条记录
   ✅ 行数匹配: 121
   ➕ 添加字段: ['课程网址']
     📝 添加: 课程网址
   ✅ 成功添加 1 个字段
   📊 非空值统计: 课程网址: 62/121
----------------------------------------

📁 处理文件夹: 面试
🎯 添加字段: ['面试']
   📄 发现CSV文件: 面试需求_处理结果_newp.csv
   📖 读取文件...
   📊 找到 121 条记录
   ✅ 行数匹配: 121
   ➕ 添加字段: ['面试']
     📝 添加: 面试
   ✅ 成功添加 1 个字段
   📊 非空值统计: 面试: 91/121
----------------------------------------

📁 处理文件夹: 申请费减免
🎯 添加字段: ['申请费减免']
   📄 发现CSV文件: 申请费减免_gemini-2.5-flash_0_121_processed.csv
   📖 读取文件...
   📊 找到 121 条记录
   ✅ 行数匹配: 121
   ➕ 添加字段: ['申请费减免']
     📝 添加: 申请费减免
   ✅ 成功添加 1 个字段
   📊 非空值统计: 申请费减免: 105/121
----------------------------------------

📁 处理文件夹: 所属院系（英文）
🎯 添加字段